In [1]:
import keras
from keras.callbacks import LambdaCallback
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Dropout
from keras.layers import ConvLSTM2D, BatchNormalization, Flatten
from keras import initializers
from keras.optimizers import RMSprop

import numpy as np

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9904615915823528097
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17959671857477882033
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 1108530239647797971
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11326753997
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11145700269338454001
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [30]:
#возможные значения а11
numbers = [i for i in range(max(y_test)*2 + 1)]
print(numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [0]:
#функция для получения последовательностей из 10 элементов
def get_sequences(features, labels, n_elem=10):
    data = []
    seqs = []
    for i in range(len(features)):
        if len(features[i:i+n_elem]) == 10:
          data.append(np.array(features[i:i+n_elem]))
          seqs.append(np.array(labels[i:i+n_elem]))
        else:
          break
    data = np.array(data)
    seqs = np.array(seqs)
    return data, seqs
#получение элемента а11
def get_elems_11(labels):
    y = []
    for i in range(len(labels)):
        y.append(np.sum(labels[i, :2]))
    y = np.array(y)
    return y

In [0]:
x_train, y_train = get_sequences(x_train, y_train)
x_test, y_test = get_sequences(x_test, y_test)

In [0]:
y_seqs_train, y_seqs_test = y_train, y_test 

In [0]:
y_train = get_elems_11(y_train)
y_test = get_elems_11(y_test)

In [35]:
print(y_train)

[ 5  4  5 ... 15 17 11]


In [0]:
#one-hot encoding
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [0]:
#input
n_seqs = x_train.shape[0]
n_elems = x_train.shape[1]
rows = x_train.shape[2]
cols = x_train.shape[3]

#convolution
filters_conv_1 = 16
filters_conv_2 = 32

#LSTM
lstm_output_size = 128

#other hyperparameters
batch_size = 64
epochs = 60
learning_rate = 1e-4

In [0]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], x_train.shape[3], 1)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], x_test.shape[3], 1)
y_test = y_test.reshape(y_test.shape[0], y_test.shape[1])

In [0]:
x = Input(shape=(n_elems, rows, cols, 1))

lstm = ConvLSTM2D(filters=40, kernel_size=(3, 3), padding='same', data_format='channels_last', return_sequences=True)(x)
drop_pred = Dropout(0.2)(lstm)
x_f = Flatten()(drop_pred)
prediction = Dense(len(numbers), activation='softmax')(x_f)
model = Model(x, prediction)

In [0]:
rmsprop = RMSprop(lr=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [0]:
ind_test = [i for i in range(len(x_test))]

In [0]:
def on_epoch_end(epoch, _):
    print()
    print('----- Predicting a11 after Epoch: %d' % epoch)
    preds_first = model.predict(x_test[0].reshape(1, n_elems, rows, cols, 1))
    ind = np.random.choice(ind_test)
    preds_rand = model.predict(x_test[ind].reshape(1, n_elems, rows, cols, 1))
    print('----- First sequence ', y_seqs_test[0])
    print('----- Prediction for the first sequence ', np.argmax(preds_first))
    print('----- Random sequence', y_seqs_test[ind])
    print('----- Prediction for the random sequence ', np.argmax(preds_rand))

In [45]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[print_callback])

scores = model.evaluate(x_test, y_test, verbose=0)

Train on 59991 samples, validate on 9991 samples
Epoch 1/60
59991/59991 [==============================] - 311s 5ms/step - loss: 2.3295 - acc: 0.2176 - val_loss: 2.3019 - val_acc: 0.2187

----- Predicting a11 after Epoch: 0
----- First sequence  [7 2 1 0 4 1 4 9 5 9]
----- Prediction for the first sequence  7
----- Random sequence [6 1 0 4 3 1 6 1 9 0]
----- Prediction for the random sequence  7
Epoch 2/60
59991/59991 [==============================] - 311s 5ms/step - loss: 2.1569 - acc: 0.2867 - val_loss: 2.2081 - val_acc: 0.2561

----- Predicting a11 after Epoch: 1
----- First sequence  [7 2 1 0 4 1 4 9 5 9]
----- Prediction for the first sequence  7
----- Random sequence [7 1 0 8 7 7 5 8 5 3]
----- Prediction for the random sequence  7
Epoch 3/60
59991/59991 [==============================] - 311s 5ms/step - loss: 2.0251 - acc: 0.3460 - val_loss: 2.1376 - val_acc: 0.2760

----- Predicting a11 after Epoch: 2
----- First sequence  [7 2 1 0 4 1 4 9 5 9]
----- Prediction for the first s

KeyboardInterrupt: ignored

In [69]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[print_callback])

scores = model.evaluate(x_test, y_test, verbose=0)

Train on 59991 samples, validate on 9991 samples
Epoch 1/60
59991/59991 [==============================] - 315s 5ms/step - loss: 0.0308 - acc: 0.9971 - val_loss: 2.2921 - val_acc: 0.4604

----- Predicting a11 after Epoch: 0
----- First sequence  [7 2 1 0 4 1 4 9 5 9]
----- Prediction for the first sequence  9
----- Random sequence [7 6 6 0 1 4 0 6 9 8]
----- Prediction for the random sequence  13
Epoch 2/60
59991/59991 [==============================] - 313s 5ms/step - loss: 0.0206 - acc: 0.9986 - val_loss: 2.3241 - val_acc: 0.4620

----- Predicting a11 after Epoch: 1
----- First sequence  [7 2 1 0 4 1 4 9 5 9]
----- Prediction for the first sequence  9
----- Random sequence [2 6 6 0 4 5 4 1 3 8]
----- Prediction for the random sequence  8


FileNotFoundError: ignored

In [0]:
model.save('my_model_2.h5')